## Setup
This section is responsible for loading the necessary libraries and initializing the OCR (Optical Character Recognition) system.

In [51]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

import os
import time
import json
import cv2
import easyocr
import numpy as np
from datetime import datetime
import requests
import warnings
import re
from dateutil.relativedelta import relativedelta
from difflib import SequenceMatcher

# Ignoring specific FutureWarning related to weights_only=False
warnings.filterwarnings("ignore", category=FutureWarning, message=".*weights_only=False.*")

# Define a function to load sample images
def load_images(image_dir):
    valid_extensions = ('.jpg', '.jpeg', '.png')
    return [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(valid_extensions)]

# Load sample images
image_dir = './sample_images'
sample_images = load_images(image_dir)

True
0
NVIDIA GeForce RTX 3060 Laptop GPU


## Format and Sort Dates
This section defines a function to format and sort dates extracted from OCR results.

In [52]:
# Function to format and sort dates
def format_and_sort_dates(dates):
    # Mapping of month abbreviations to numbers
    month_conversion = {
        "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12"
    }
    
    formatted_dates = []
    for date in dates:
        # Removing non-alphanumeric characters
        date = re.sub(r'[^a-zA-Z0-9]', ' ', date)
        # Converting month abbreviations to numbers
        for month in month_conversion:
            date = re.sub(month, month_conversion[month], date, flags=re.IGNORECASE)
        # Reformatting the date string
        date = re.sub(r'\s+', '/', date).strip()
        formatted_dates.append(datetime.strptime(date, "%d/%m/%Y"))
    
    # Sorting the dates
    formatted_dates.sort()
    return formatted_dates

## Extract Dates
This section defines a function to extract dates from OCR results.

In [53]:
# Function to extract dates from OCR results
def extract_dates(results):
    month_abbrs = r'(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)'
    
    # Various date patterns to match
    date_patterns = [
        r'\b\d{2}/\d{2}/\d{4}\b',  # e.g., 31/12/2024
        r'\b\d{2}-\d{2}-\d{4}\b',  # e.g., 31-12-2024
        r'\b\d{4}-\d{2}-\d{2}\b',  # e.g., 2024-12-31
        r'\b\d{4}/\d{2}/\d{2}\b',  # e.g., 2024/12/31
        r'\b\d{2} ' + month_abbrs + r' \d{4}\b',  # e.g., 31 JAN 2024
        r'\b\d{2}-' + month_abbrs + r'-\d{4}\b',  # e.g., 31-JAN-2024
        r'\b\d{2}-' + month_abbrs + r'-\d{2}\b',  # e.g., 31-JAN-24
        r'\b\d{2}.\d{2}.\d{4}\b',  # e.g., 31.12.2024
        r'\b\d{2}.\d{2}.\d{2}\b',  # e.g., 31.12.24
        r'\b\d{2} \w{3} \w{3} \d{2}\b',  # e.g., 31 JAN JAN 24
        r'\b\d{2} ' + month_abbrs + ' ' + month_abbrs + r' \d{2}\b',  # e.g., 31 JAN /JAN 24
        r'\b\d{2} ' + month_abbrs + '/' + month_abbrs + r' \d{2}\b',  # e.g., 31 JAN/JAN 24
        r'\b\d{2} ' + month_abbrs + r' \d{2}\b',  # e.g., 31 JAN 24
    ]
    
    dates = set()
    
    # Searching for date patterns in OCR results
    for (_, text, _) in results:
        for pattern in date_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                dates.add(match)
    
    return list(dates)

## Correct Date with Month Search
This section defines a function to correct dates with month abbreviations.

In [54]:
# Function to correct date using month search
def correct_date_with_month_search(date_str, text_lines, date_type):
    # Mapping of month abbreviations to numbers
    month_conversion = {
        "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12"
    }
    
    months_found = set()
    # Searching for months in the text lines
    for _, line, _ in text_lines:
        line_upper = line.upper()
        for month in month_conversion:
            if re.search(r'\b' + month + r'\b', line_upper):
                months_found.add(month)
    
    # If enough months are found, correct the date string
    if len(months_found) >= 3:
        if date_type == 'birth':
            month = months_found[0]
        elif date_type == 'expiry':
            month = months_found[-1]
        corrected_date = re.sub(r'[^0-9]', '', date_str)
        corrected_date = corrected_date[:2] + month_conversion[month] + corrected_date[4:]
    else:
        # Apply character corrections if months are not found
        corrections = {
            'A': '1', 'B': '8', 'C': '0', 'D': '0', 'E': '3', 'F': '7', 'G': '6',
            'H': '4', 'I': '1', 'J': '1', 'K': '1', 'L': '1', 'M': '0', 'N': '0',
            'O': '0', 'P': '9', 'Q': '0', 'R': '2', 'S': '5', 'T': '7', 'U': '0',
            'V': '8', 'W': '8', 'X': '8', 'Y': '4', 'Z': '2'
        }
        corrected_date = ''.join(corrections.get(c, c) for c in date_str)
    
    return corrected_date

## Correct Passport Number
This section defines a function to correct common OCR errors in the passport number.

In [55]:
# Function to correct passport number
def correct_passport_number(passport_number):
    corrections = {
        'O': '0',
        'I': '1',
        'B': '8',
        'G': '4'
    }
    corrected_passport_number = list(passport_number)
    # Applying character corrections to the passport number
    for i, char in enumerate(corrected_passport_number):
        if i > 0 and char in corrections:
            corrected_passport_number[i] = corrections[char]
    return ''.join(corrected_passport_number)

## Fetch ICAO Codes & Country Data
This section defines a function to fetch data, both ICAO country codes and nationalities, from an online public API.

In [56]:
# Function to fetch country data
def fetch_data():
    url = "https://restcountries.com/v3.1/all"
    response = requests.get(url)
    data = response.json()
    
    icao_codes = set()
    for country in data:
        if "cca3" in country:  
            icao_codes.add(country["cca3"].upper())

    country_code_to_nationality = {}
    for country in data:
        if "cca3" in country and "name" in country:
            cca3 = country["cca3"].upper()
            nationality = country["name"]["official"].capitalize()
            country_code_to_nationality[cca3] = nationality
    
    return icao_codes, country_code_to_nationality

## Levenshtein Distance
This section defines a function to calculate the Levenshtein distance between two strings.

In [57]:
# Function to calculate Levenshtein distance
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

## Correct Nationality
This section defines a function to correct the nationality field in the OCR results.

In [58]:
# Function to correct nationality based on text lines
def correct_nationality(nationality, text_lines):
    corrections = {
        '0': 'O',
        '1': 'I',
        '2': 'Z',
        '5': 'S',
        '8': 'B'
    }
    corrected_nationality = list(nationality)
    # Applying character corrections to the nationality
    for i, char in enumerate(corrected_nationality):
        if char in corrections:
            corrected_nationality[i] = corrections[char]
    corrected_nationality_str = ''.join(corrected_nationality)

    valid_country_codes, _ = fetch_data()
    
    if corrected_nationality_str not in valid_country_codes:
        all_texts = []
        # Collecting all text lines for comparison
        for _, line, _ in text_lines:
            all_texts.extend(line.split())
        
        min_score = float('inf')
        closest_match = corrected_nationality_str
        for text in all_texts:
            score = levenshtein_distance(corrected_nationality_str, text.upper())
            if score < min_score:
                min_score = score
                closest_match = text.upper()
        
        return closest_match
    
    return corrected_nationality_str

## Preprocess the Image
This cell contains a function to preprocess the image before performing OCR.

In [59]:
# Preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    
    # Increase image resolution by resizing
    scale_percent = 200  # Increase size by 200%
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation=cv2.INTER_CUBIC)
    
    # Converting the image to grayscale for better analysis
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # Apply adaptive thresholding to get a binary image
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 17, 5)
    
    return binary

## Perform OCR using EasyOCR
This cell defines a function to perform OCR using the EasyOCR library.

In [60]:
# Function to extract text from the image using OCR
def extract_text(image_path):
    image = preprocess_image(image_path)
    results = reader.readtext(image)
    
    mrz_lines = []
    # Extracting lines of text that are likely part of the MRZ (longer texts)
    for (_, text, _) in results:
        if len(text) >= 15:  
            mrz_lines.append(text)
    
    return results, mrz_lines[-2:], image

## Print Identified Text and Parse MRZ Lines
This cell contains functions to print identified text with bounding boxes, parse MRZ lines, and calculate sequence similarity.

In [61]:
# Print all identified text with their bounding boxes
def print_identified_text(results):
    for (bbox, text, prob) in results:
        print(f"Text: '{text}', Probability: {prob}, Bounding Box: {bbox}")

# Function to parse MRZ lines to extract passport fields
def parse_mrz(mrz_lines, text_lines):
    fields = {}
    # Cleaning MRZ lines to keep only alphanumeric characters and '<'
    mrz_lines = [''.join(c if c.isalnum() or c == '<' else '' for c in line) for line in mrz_lines]
    
    # Extracting name information from the MRZ
    if '<<' in mrz_lines[0]:
        name_start_index = mrz_lines[0].find('<<') + 2
    else:
        name_start_index = mrz_lines[0].find('<') + 1
    
    name_data_raw = mrz_lines[0][name_start_index:44]
    name_data = [entry for entry in name_data_raw.split('<') if entry and len(entry) > 1]
    fields['name'] = ' '.join(name_data).replace('<', ' ').strip().upper()
    
    # Extracting and formatting date of birth
    date_of_birth_raw = mrz_lines[1][13:19]
    date_of_birth_corrected = correct_date_with_month_search(date_of_birth_raw, text_lines, 'birth')
    date_of_birth_formatted = f"{date_of_birth_corrected[4:6]}/{date_of_birth_corrected[2:4]}/19{date_of_birth_corrected[0:2]}"
    fields['date_of_birth'] = date_of_birth_formatted

    # Extracting and formatting date of expiry
    date_of_expiry_raw = mrz_lines[1][21:27]
    date_of_expiry_corrected = correct_date_with_month_search(date_of_expiry_raw, text_lines, 'expiry')
    date_of_expiry_formatted = f"{date_of_expiry_corrected[4:6]}/{date_of_expiry_corrected[2:4]}/20{date_of_expiry_corrected[0:2]}"
    fields['date_of_expiry'] = date_of_expiry_formatted

    # Fetching nationality information
    _, country_code_to_nationality = fetch_data()
    nationality = mrz_lines[1][10:13].upper()
    corrected_nationality = correct_nationality(nationality, text_lines)
    fields['nationality'] = country_code_to_nationality.get(corrected_nationality, corrected_nationality).upper()

    # Extracting passport type
    fields['passport_type'] = re.sub('[^A-Za-z]', '', mrz_lines[0][0:2]).upper()
    
    # Extracting and correcting passport number
    passport_number = mrz_lines[1][0:9].replace('<', '').upper()
    fields['passport_number'] = correct_passport_number(passport_number)
    fields['date_of_issue'] = ''
    fields['authority'] = ''

    return fields

# Function to calculate sequence similarity between two strings
def sequence_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

## Extract Authority Field
This cell contains a function to extract the authority field name and bounding box from the OCR results.

In [62]:
# Function to extract issuing authority from OCR results
def extract_authority(results):
    # List of keywords related to issuing authority
    authority_keywords = ['Place of Issue', 'Authority', 'Place of Issuance', 'Issued by', 'Issuing Office', 'Code of Issuing', 'Issuing Country', 'Issuing Authority']
    max_similarity = 0
    authority_bbox = None
    authority_text = ""

    # Searching for the most similar text to authority keywords
    for (bbox, text, prob) in results:
        for keyword in authority_keywords:
            similarity = sequence_similarity(text.lower(), keyword.lower())
            if similarity > max_similarity:
                max_similarity = similarity
                authority_bbox = bbox
                authority_text = text
    
    if authority_bbox is None:
        return "Authority not found", None
    
    return authority_text, authority_bbox

## Generate Points on Edges and Find Closest Text

This cell contains two functions, both to generate a specified number of points along the edges of a bounding box to then be able to find the closest text to a given bounding box from a set of OCR results.

In [63]:
# Function to generate points on the edges of a bounding box
def generate_points_on_edges(bbox, num_points=10):
    points = []

    for i in range(len(bbox)):
        start = np.array(bbox[i])
        end = np.array(bbox[(i + 1) % len(bbox)])
        edge_points = [start + t * (end - start) for t in np.linspace(0, 1, num_points)]
        points.extend(edge_points)

    return points

# Function to find the closest text to a given bounding box
def find_closest_text(bbox, results, exclude_bbox):
    min_distance = float('inf')
    closest_text = ""

    # Generate points on the edges of the given bounding box
    bbox_points = generate_points_on_edges(bbox)

    for (other_bbox, text, prob) in results:
        # Skip the bounding box that needs to be excluded and those with low probability
        if other_bbox == exclude_bbox or prob <= 0.5:
            continue
        # Generate points on the edges of the current bounding box in results
        other_bbox_points = generate_points_on_edges(other_bbox)

        current_min_distance = float('inf')

        # Calculate the distance between each point of the two bounding boxes
        for bbox_point in bbox_points:
            for other_bbox_point in other_bbox_points:
                distance = np.linalg.norm(bbox_point - other_bbox_point) # Euclidean distance
                # Update the current minimum distance if a smaller distance is found
                if distance < current_min_distance:
                    current_min_distance = distance

        # Update the overall minimum distance and closest text if a smaller distance is found
        if current_min_distance < min_distance:
            min_distance = current_min_distance
            closest_text = text

    return closest_text

## Main Function to Process Image and Extract Information
This cell defines the main function main(image_path) to process an image and extract relevant information from it.

In [64]:
# Main function to process the image and extract relevant fields
def main(image_path):
    results, mrz_lines, preprocessed_image = extract_text(image_path)
    
    # If MRZ lines are detected, parse them to extract fields
    if len(mrz_lines) >= 2:  
        fields = parse_mrz(mrz_lines[:2], results)  
    else:
        print("MRZ not fully detected")
        fields = {}

    dates = extract_dates(results)

    # Format and sort dates
    formatted_dates = format_and_sort_dates(dates)
    
    # Handling different cases for date fields
    if len(formatted_dates) == 3:
        fields['date_of_issue'] = formatted_dates[1].strftime("%d/%m/%Y")
        fields['date_of_birth'] = formatted_dates[0].strftime("%d/%m/%Y")
        fields['date_of_expiry'] = formatted_dates[2].strftime("%d/%m/%Y")
    else:
        # Infer date_of_issue if not found
        if 'date_of_issue' not in fields or not fields['date_of_issue']:
            if 'date_of_expiry' in fields:
                try:
                    date_of_expiry_date = datetime.strptime(fields['date_of_expiry'], "%d/%m/%Y")
                    date_of_issue_date = date_of_expiry_date - relativedelta(years=10)
                    fields['date_of_issue'] = date_of_issue_date.strftime("%d/%m/%Y")
                except ValueError as e:
                    print(f"Date parsing error: {e}")
                    fields['date_of_issue'] = '00/00/00'

    _, authority_bbox = extract_authority(results)
    
    if authority_bbox:
        authority_value = find_closest_text(authority_bbox, results, authority_bbox)
        fields['authority'] = authority_value

   # Create a list to hold the text and bounding box information
    output_data = []

    # Iterate over results to extract text and bounding box information
    for (bbox, text, _) in results:
        bbox = [list(map(int, point)) for point in bbox]
        output_data.append({
            "text": text,
            "bounding_box": bbox
        })

    # Convert the list to a JSON formatted string
    json_output = json.dumps(output_data, indent=4)

    print(json_output)
        
    return fields, output_data

## Functions to Evaluate OCR Accuracy and Run Tests
This cell defines functions to evaluate OCR accuracy and run tests.

In [65]:
# Function to evaluate OCR accuracy
def evaluate_ocr_accuracy(image_path, ground_truth):
    """
    Evaluates the accuracy of OCR on a given image by comparing the extracted fields to ground truth data.

    Parameters:
    - image_path: Path to the image file to be processed.
    - ground_truth: Dictionary containing the ground truth data for comparison.

    Returns:
    - accuracy: The accuracy of the OCR as a ratio of correctly extracted fields.
    - processing_time: Time taken to process the image.
    - fields: Extracted fields from the image.
    """
    start_time = time.time()  # Start the timer for processing time
    fields, _ = main(image_path)  # Extract fields from the image using the main function
    processing_time = time.time() - start_time  # Calculate the time taken for processing

    # Calculate accuracy
    correct_fields = 0
    for key in ground_truth:
        if key in fields:  # Check if the key exists in the extracted fields
            if isinstance(ground_truth[key], list):
                if fields[key] in ground_truth[key]:
                    correct_fields += 1
            else:
                if fields[key] == ground_truth[key]:
                    correct_fields += 1

    accuracy = correct_fields / len(ground_truth)  # Calculate the accuracy as a ratio
    return accuracy, processing_time, fields

# Function to run tests on multiple sample images
def run_tests(sample_images, ground_truth_data):
    """
    Runs OCR accuracy tests on multiple sample images and prints the results.

    Parameters:
    - sample_images: List of paths to sample image files.
    - ground_truth_data: Dictionary containing ground truth data for each image.

    Returns:
    - None
    """
    results = [] 
    for image_path in sample_images:
        image_name = os.path.basename(image_path)  
        if image_name in ground_truth_data:  # Check if ground truth data exists for the image
            ground_truth = ground_truth_data[image_name] 
            accuracy, processing_time, fields = evaluate_ocr_accuracy(image_path, ground_truth)  # Evaluate OCR accuracy
            # Store the results in a dictionary
            results.append({
                "image": image_name,
                "accuracy": accuracy,
                "processing_time": processing_time,
                "fields": fields,
                "ground_truth": ground_truth
            })
    
    # Print the results for each image
    for result in results:
        print(f"Image: {result['image']}")
        print(f"Accuracy: {result['accuracy'] * 100}%")
        print(f"Processing Time: {result['processing_time']} seconds")
        print("Extracted Fields:")
        for key, value in result['fields'].items():
            print(f"  {key}: {value}")
        print("Ground Truth:")
        for key, value in result['ground_truth'].items():
            print(f"  {key}: {value}")
        print("\n")

# Function to load test data from a JSON file
def load_test(file_path):
    """
    Loads test data from a JSON file, in order to be able to measure its accuracy.

    Parameters:
    - file_path: Path to the JSON file containing test data.

    Returns:
    - data: Dictionary containing the test data.
    """
    with open(file_path, 'r') as f:
        return json.load(f)  

## Initialize and Run Tests
This cell initializes the EasyOCR reader, loads the ground truth data, and runs the tests.

In [66]:
# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True) 

# Load the ground truth data from the JSON file
ground_truth_data = load_test('ground_truth_data.json')

# Run the tests and collect results
run_tests(sample_images, ground_truth_data)

[
    {
        "text": "Sottt",
        "bounding_box": [
            [
                453,
                0
            ],
            [
                591,
                0
            ],
            [
                591,
                20
            ],
            [
                453,
                20
            ]
        ]
    },
    {
        "text": "Mes",
        "bounding_box": [
            [
                654,
                2
            ],
            [
                780,
                2
            ],
            [
                780,
                34
            ],
            [
                654,
                34
            ]
        ]
    },
    {
        "text": "0 2 ' ,",
        "bounding_box": [
            [
                1107,
                359
            ],
            [
                1252,
                359
            ],
            [
                1252,
                400
            ],
            [
                1107

## Load Sample Images and Run Tests with Larger Dataset
This cell loads a larger set of sample images that weren't taken into account when implementing the OCR system, and runs the tests with a larger ground truth dataset.

In [67]:
# Load sample images
image_dir = './images'
test_images = load_images(image_dir)

# Load the ground truth data from the JSON file
big_ground_truth_data = load_test('big_ground_truth_data.json')

# Run the tests and collect results
run_tests(test_images, big_ground_truth_data)

[
    {
        "text": "603",
        "bounding_box": [
            [
                399,
                67
            ],
            [
                471,
                67
            ],
            [
                471,
                109
            ],
            [
                399,
                109
            ]
        ]
    },
    {
        "text": "AUSTRALIA",
        "bounding_box": [
            [
                575,
                67
            ],
            [
                773,
                67
            ],
            [
                773,
                107
            ],
            [
                575,
                107
            ]
        ]
    },
    {
        "text": "DIPLOMATIC",
        "bounding_box": [
            [
                39,
                99
            ],
            [
                257,
                99
            ],
            [
                257,
                141
            ],
            [
           

<i>*Note that, in order to test the robustness of the system, you should count on a json file with the ground truth data such as the ones already included in the project, and change the `image_dir` to include the new pictures that will take part of the test, or check it manually following the instructions in the README file on how to run the Streamlit app to make the process smoother.


In this assessment, I have been working on developing an OCR (Optical Character Recognition) system to accurately extract information from passport images. The system uses the EasyOCR library to recognize text from images, processes the extracted text, and matches it against predefined fields, such as name, date of birth, nationality, passport number, date of issue, date of expiry, and issuing authority.

### Detailed Workflow

1. **Image Preprocessing**: To enhance the image quality to improve OCR accuracy.
   - **Steps**:
     - Load the image using OpenCV.
     - Increase the image resolution by resizing it.
     - Convert the image to grayscale.
     - Apply Gaussian blur to reduce noise.
     - Apply adaptive thresholding to create a binary image.

2. **Text Extraction using EasyOCR**: To extract text from the preprocessed image.
   - **Steps**:
     - Use EasyOCR to read text from the image.
     - Filter results to focus on the Machine Readable Zone (MRZ) which contains the most important information in passports, and perhaps an approach due to the time constraint a bit more efficient.

3. **Parsing MRZ Lines**: To extract structured data from MRZ lines.
   - **Steps**:
     - Clean and parse the MRZ lines to extract fields like name, date of birth, date of expiry, nationality, passport type, and passport number.
     - Correct common OCR errors and format the extracted data appropriately.

4. **Date Extraction and Formatting**: To extract and format dates found in the OCR results.
   - **Steps**:
     - Use regular expressions to find date patterns in the text.
     - Convert these dates to a standard format and sort them.

5. **Field Extraction**: To extract specific fields from the OCR results.
   - **Steps**:
     - Extract the authority field by identifying keywords related to issuing authorities.
     - Find the closest text to the identified authority bounding box for accurate extraction.

6. **Evaluation of OCR Accuracy**: To finally evaluate the accuracy of the OCR system by comparing extracted fields with ground truth data.
   - **Steps**:
     - Calculate the processing time.
     - Compare each extracted field with the corresponding ground truth field to compute the accuracy.
     - Aggregate results and print the performance metrics.

### Results and Metrics

#### Image: Canada.jpg
- **Accuracy**: 100.0%
- **Processing Time**: 1.50 seconds
- **Extracted Fields**:
  - Name: CHRISTINE EDITH
  - Date of Birth: 07/11/1959
  - Date of Expiry: 26/01/2028
  - Nationality: CANADA
  - Passport Type: P
  - Passport Number: AE021527
  - Date of Issue: 26/01/2018
  - Authority: GATINEAU
- **Ground Truth**: Matches exactly with the extracted fields.

#### Image: Iceland.jpg
- **Accuracy**: 87.5%
- **Processing Time**: 2.60 seconds
- **Extracted Fields**:
  - Name: THURIDUR OESP
  - Date of Birth: 12/12/1912
  - Date of Expiry: 10/03/2031
  - Nationality: ICELAND
  - Passport Type: PA
  - Passport Number: A3536444
  - Date of Issue: 10/03/2021 (Extracted as 10/03/2021, but ground truth is 12/03/2021)
  - Authority: PJODSKRA ISLANDS
- **Ground Truth**: Matches except for the date of issue.

#### Image: India.jpg
- **Accuracy**: 100.0%
- **Processing Time**: 3.07 seconds
- **Extracted Fields**:
  - Name: SITA MAHA LAKSHMI
  - Date of Birth: 23/09/1959
  - Date of Expiry: 10/10/2021
  - Nationality: REPUBLIC OF INDIA
  - Passport Type: P
  - Passport Number: J8369854
  - Date of Issue: 11/10/2011
  - Authority: HYDERABAD
- **Ground Truth**: Matches exactly with the extracted fields.

#### Image: Indonesia.jpg
- **Accuracy**: 87.5%
- **Processing Time**: 4.67 seconds
- **Extracted Fields**:
  - Name: NAMA
  - Date of Birth: 17/08/1945
  - Date of Expiry: 26/01/2016
  - Nationality: REPUBLIC OF INDONESIA
  - Passport Type: P
  - Passport Number: X000000
  - Date of Issue: 26/01/2006 (Extracted as 26/01/2006, but ground truth is 26/01/2011)
  - Authority: KANTOR IMIGRASI
- **Ground Truth**: Matches except for the date of issue.

#### Image: Kyrgyzstan.jpg
- **Accuracy**: 87.5%
- **Processing Time**: 1.25 seconds
- **Extracted Fields**:
  - Name: USON
  - Date of Birth: 09/12/2000
  - Date of Expiry: 09/12/2020
  - Nationality: KYRGYZ REPUBLIC
  - Passport Type: PD
  - Passport Number: PE0000000
  - Date of Issue: 09/08/2019
  - Authority: ASANOV (Extracted as ASANOV, but ground truth is SRS 218000)
- **Ground Truth**: Matches except for the authority field.

Getting a final accuracy of 92.5% - performing pretty well.

### Implementation Insights
* **Hardware Utilization**: One of the key factors that made the process smoother was leveraging GPU acceleration. Transitioning from macOS to an MSI laptop enabled the use of CUDA, significantly enhancing the processing speed and efficiency of the OCR tasks.
* **Binary Image Inversion**: Another crucial improvement was inverting the binary image colors during preprocessing. This inversion proved to be beneficial for text recognition accuracy - actually to improve accuracy on low-contrast images.

### Analysis and Conclusion
* **High Accuracy**: The OCR system demonstrated a high level of accuracy in extracting key fields from passport images. Notably, the system achieved a perfect accuracy of 100% for the images Canada.jpg and India.jpg, indicating its capability to correctly identify and extract all relevant information for these cases.
* **Common Issues**: Despite the overall strong performance, minor discrepancies were observed in certain fields. For instance, the date of issue was incorrectly extracted for Iceland.jpg and Indonesia.jpg. Additionally, the authority field for Kyrgyzstan.jpg did not match the ground truth. The authority field proved to be the most challenging to analyze due to its non-redundant nature and the difficulty of inferring it from other data.
* **Processing Time**: The system's processing times varied depending on the image, ranging from approximately 1.25 seconds to 4.67 seconds. This demonstrates the efficiency of the OCR tasks, facilitated significantly by GPU acceleration, which enhanced the overall speed and performance of the system.

Overall, the OCR system exhibits a robust ability to accurately extract and identify passport details, though there is room for improvement in handling specific fields such as the date of issue and the authority. This project has been highly insightful and rewarding, providing numerous learning opportunities. Key improvements, such as utilizing GPU acceleration and inverting binary image colors, contributed significantly to the system's performance. Given more time, further refinements could be implemented to enhance accuracy and reliability, particularly in the extraction of complex fields like the authority.